In [22]:
import re

import pandas as pd
import numpy as np

from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

In [23]:
lamah_ce_path = Path("/home/wuhlmann/BA/data/raw_data/LamaH-CE_daily")

#### Confirm integretity for climatic timeseries, hydrologic indices and static attributes. 

In [30]:
B_climatic_timeseries_path = lamah_ce_path / "B_basins_intermediate_all/2_timeseries/daily"
C_climatic_timeseries_path = lamah_ce_path / "C_basins_intermediate_lowimp/2_timeseries/daily"
hydrologic_signatures_path = lamah_ce_path / "D_gauges/1_attributes/Hydro_indices_1981_2017.csv"

B_timeseries_paths = list(sorted(B_climatic_timeseries_path.glob("*.csv")))
C_timeseries_paths = list(sorted(C_climatic_timeseries_path.glob("*.csv")))

# Get IDs of low impact basins. 
C_ids = []
for p in C_timeseries_paths:
    stem = Path(p).stem
    match = re.search(r'\d+', stem)
    if match:
        C_ids.append(int(match.group()))
C_ids = sorted(C_ids)

Climatic values do not contain any NA, as expected from ERA5 data. 

In [28]:
for path in tqdm(B_timeseries_paths):

    ts = pd.read_table(path, sep=";", header=0)

    if ts.isnull().values.any(): 
        print(f"Climatic data {path.stem} contains NA")

100%|██████████| 859/859 [00:24<00:00, 34.97it/s]


17/9 Basins contain NA in their hydrologic indices, 4/2 of those only at the baseflow indices, if they are not used as predictor, still usable. Discard other 13/7 catchments. 

In [40]:
hydrologic_signatures = pd.read_table(hydrologic_signatures_path, header=0, sep=";")

A_B_basins_with_NA = hydrologic_signatures[hydrologic_signatures.isna().any(axis=1)]
print(A_B_basins_with_NA)

C_hydrologic_signatures = hydrologic_signatures[hydrologic_signatures["ID"].isin(C_ids)]

C_basins_with_NA = C_hydrologic_signatures[C_hydrologic_signatures.isna().any(axis=1)]
#C_basins_with_NA

      ID  q_mean  runoff_ratio  stream_elas  slope_fdc  baseflow_index_ladson  \
53    55     NaN           NaN          NaN        NaN                    NaN   
316  319     NaN           NaN          NaN        NaN                    NaN   
349  352   0.404         0.207        1.559      2.474                    NaN   
369  376     NaN           NaN          NaN        NaN                    NaN   
399  407     NaN           NaN          NaN        NaN                    NaN   
402  410     NaN           NaN          NaN        NaN                    NaN   
406  414   0.184         0.062        5.272        NaN                  0.331   
533  551     NaN           NaN          NaN        NaN                    NaN   
535  553     NaN           NaN          NaN        NaN                    NaN   
555  573     NaN           NaN          NaN        NaN                    NaN   
607  625     NaN           NaN          NaN        NaN                    NaN   
657  675     NaN           N

In [38]:
B_static_attributes_path = lamah_ce_path / "B_basins_intermediate_all/1_attributes/Catchment_attributes.csv"
B_static_attributes = pd.read_table(B_static_attributes_path, header=0, sep=";")
B_basins_NA_attr =  B_static_attributes[B_static_attributes.isna().any(axis=1)]

C_static_attributes_path = lamah_ce_path / "C_basins_intermediate_lowimp/1_attributes/Catchment_attributes.csv"
C_static_attributes = pd.read_table(C_static_attributes_path, header=0, sep=";")
C_basins_NA_attr = C_static_attributes[C_static_attributes.isna().any(axis=1)]


In [41]:
print(len(B_basins_NA_attr), len(C_basins_NA_attr))

53 21
